# Mapa de avistamientos de animles en Argentina, dentro y fuera de áreas protegidas

## ¿De donde obtuvimos los datos?
Para esta demostración, vamos a usar datos de:
GBIF, una base de datos colaborativa de información de seres vivos. [Link](https://www.gbif.org/occurrence/map?basis_of_record=HUMAN_OBSERVATION&basis_of_record=OBSERVATION&basis_of_record=OCCURRENCE&country=AR&taxon_key=1&year=2015,*&occurrence_status=present) [Descargar](https://doi.org/10.15468/dl.qtw77x)

El mapa de argentina Argenmap. [Link](https://https://mapa.ign.gob.ar/)

Información sobre las áreas protegidas de Argentina. [Link](https://www.ign.gob.ar/NuestrasActividades/Geografia/DatosArgentina/ParquesNacionales2) [Descargar](https://dnsg.ign.gob.ar/apps/api/v1/capas-sig/Geodesia+y+demarcaci%C3%B3n/L%C3%ADmites/area_protegida/json)

## Preprocesamiento

### CSV de ocurrencias

Este archivo contiene todas las ocurrencias de observaciones de animales en Argentina después del año 2015, casi una década de observaciones! En total 11,366,706 observaciones, con todos los datos asociados. Esto naturalmente conlleva una cantidad enorme de datos 11.6 GB en el archivo de ocurrencias, pero como estos son datos crudos hay que procesarlos antes de poder trabajar. Las herramientas que usualmente utilizamos no son suficientes ya que usualmente pandas no puede cargar un archivo tan grande, por lo que tendremos que ser un poco creativos.

Pero antes de todo esto, hagamosnos una idea de como se ven nuestros datos:

In [ ]:
import pandas as pd
from pathlib import Path
import os
DATA_PATH = Path.cwd() / "datasets"
GBIF_PATH = DATA_PATH / "gbif"
OCURRENCES_PATH =  GBIF_PATH / "occurrence.txt"

Lo primero que hacemos es importar la librería de análisis de datos Pandas, con la que vamos a darle forma a nuestro dataframe, luego establecemos el path a nuestro archivo de ocurrencias con pathlib para facilitar la escritura del código a continuación y otros paths que utilizaremos luego.

In [ ]:
ocurrences = pd.read_csv(OCURRENCES_PATH, encoding="utf-8", engine='c', nrows=100, delimiter="\t")

Aquí leemos las primeras 100 columnas del archivo (con nrows) para evitar sobrecargar el sistema y poder darnos una idea de con que estamos trabajando, el archivo esta separado por tbs en lugar de comas por lo que usamos delimiter="\t" y para evitar problemas con las ñ y las tildes usamos utf8 como encoding. El engine c ayuda a acelerar el procesamiento de los datos.

In [ ]:
ocurrences.head()

El metodo head nos dio un vistazo del dataframe y hay demasiadas columnas, 223 en total, veamoslas:

In [ ]:
ocurrences.columns

Tantas columnas que no se llegan a mostrar, por suerte no las necesitamos todas, el siguiente paso es cortar las columnas innecesarias:

In [ ]:
COLUMNS = ["gbifID","catalogNumber","year","month","day","eventTime","decimalLongitude","decimalLatitude","class","order","family","verbatimScientificName","vernacularName","taxonKey","level1Name","level2Name","iucnRedListCategory"]
ocurrences = ocurrences[COLUMNS]
ocurrences.head()

Mucho mejor, las columnas elegidas nos permiten identificar la ocurrencia, cuando y donde ocurrio y cuales son los datos del animal además de su clasificación según el risgo de extinción. Aun así, ya que sabmeos que columnas queremos podríamos nunca cargralas en primer lugar:

In [ ]:
ocurrences = pd.read_csv(OCURRENCES_PATH, encoding="utf-8", engine='c', nrows=100, delimiter="\t", usecols=COLUMNS)
ocurrences.head()

Aquí podemos ver que muchas columnas dicen NaN, eso significa que contienen valores nulos, esto puede solucionarse con Dataframe.dropna, esto borra las filas con valores nulos, aunque no queremos borrar todas las columnas que por ejemplo no tengan un nombre vernacular:

In [ ]:
NO_NA_COLUMNS = ["gbifID","catalogNumber","year","month","day","decimalLongitude","decimalLatitude","class","order","family","verbatimScientificName","taxonKey","level1Name","level2Name","iucnRedListCategory"]
ocurrences.dropna(subset=NO_NA_COLUMNS, inplace=True)
ocurrences.head()

Elegimos las columnas que no pueden tener valores nulos y los borramos. Con estas filas borradas el archivo será más facil de visualizar y más pequeño.

Eso simplifica nuestro trabajo de ahora en adelante. Aunque nos ahorremos estas columnas el archivo entero sigue siendo muy pesado, por lo que vamos a tener que encontrar alguna forma de procesarlo por trozos.

In [ ]:
header_written = False
for chunk in pd.read_csv(OCURRENCES_PATH, chunksize=100000, usecols=COLUMNS, encoding="utf-8", engine='c',delimiter="\t"):
    chunk.dropna(subset=NO_NA_COLUMNS, inplace=True)
    if header_written:
        chunk.to_csv(DATA_PATH / "animals_filtered.csv",mode="a", index=False, encoding="utf-8", header=False)
    else:
        chunk.to_csv(DATA_PATH / "animals_filtered.csv",mode="w", index=False, encoding="utf-8")
        header_written = True

Hay mucho que explicar, la idea es dividir el archivo en trozos (chunks) de 100000 filas, borramos los nulos y los escribimos en un nuevo archivo llamado animals filtered. Esto tiene un giro por el header que solo debe ser escrito una vez, para eso está la variable header written que se encarga de que la primera escritura sea con header y cree el archivo si no existe y el resto solo agreguen los trozos al final.
Finalmente con este procesamiento el archivo queda con la información necesaria y puede ser usado para las visualizaciones.

### CSV multimedia

Este CSV que se encuentra en la descarga de GBIF contiene links a imágenes de varias de las observaciones que nos encontramos en el archivo de ocurrencias. Lo que queremos es asociar estos links mediante el gbifid (que se encuentra en ambos datasets) para tener un dataset unificado. 

In [ ]:
MULTIMEDIA_PATH = GBIF_PATH / "multimedia.txt"
multimedia = pd.read_csv(MULTIMEDIA_PATH, encoding="utf-8", engine='c', nrows=100, delimiter="\t")
multimedia.head()

De vuelta no queremos todas las columnas, solo nos interesa el link y el gbifid

In [ ]:
multimedia.columns

In [ ]:
multimedia = pd.read_csv(MULTIMEDIA_PATH, encoding="utf-8", engine='c', nrows=100, delimiter="\t", usecols=["gbifID","format","identifier"])
multimedia = multimedia[multimedia["format"] == "StillImage"]
multimedia.dropna(subset=["gbifID"], inplace=True)
multimedia.rename(columns={"identifier":"image"}, inplace=True)
multimedia.head()

Dejamos solo las imágenes, borramos las filas sin id y cambiamos el nombre de la columna identifier por uno más entendible.
Con las columnas que queremos procesamos el ocurrences nuevo con el archivo de multimedia:

In [ ]:
header_written = False
for chunk in pd.read_csv(OCURRENCES_PATH, chunksize=100000, usecols=COLUMNS, encoding="utf-8", engine='c',delimiter="\t"):
    merged_df = pd.merge(chunk, multimedia, on="gbifID", how="left")
    if header_written:
        merged_df.to_csv(DATA_PATH / "animals_multimedia.csv",mode="a", index=False, encoding="utf-8", header=False)
    else:
        merged_df.to_csv(DATA_PATH / "animals_multimedia.csv",mode="w", index=False, encoding="utf-8")
        header_written = True

El bloque anterior es muy similar al de ocurrences, la diferencia es la operacion principal, en este caso es un left merge, este combina los dataframes donde coincidan los gbifids y si no hay un match, mantiene la fila de ocurrences, luego todo se guarda en animals_multimedia.csv

## Mapa de ocurrencias

En esta parte se genera el mapa con las ocurrencias, dado el tamaño de los datos esto usualmente toma mucho tiempo, por esto solo obtenemos ciertas vistas como por ejemplo los animales en peligro de extinción.

In [3]:
from shapely.geometry import Point
import geopandas as gpd
import plotly.express as px
from pathlib import Path
import pandas as pd

BASE_PATH = Path.cwd()
DATA_PATH = BASE_PATH / "datasets"

AREAS_PATH = DATA_PATH / "area_protegida.json"
ANIMALS_PATH = DATA_PATH / "animals_multimedia.csv"

COLS = [
    "decimalLatitude",
    "decimalLongitude",
    "vernacularName",
    "verbatimScientificName",
    "iucnRedListCategory",
    "image",
]
TYPES = {
    "decimalLatitude": float,
    "decimalLongitude": float,
    "vernacularName": str,
    "verbatimScientificName": str,
    "iucnRedListCategory": str,
    "image": str,
}
ARGENMAP_STYLE = {
    "version": 8,
    "sources": {
        "argenmap": {
            "type": "raster",
            "scheme": "tms",
            "tiles": [
                "https://wms.ign.gob.ar/geoserver/gwc/service/tms/1.0.0/capabaseargenmap@EPSG%3A3857@png/{z}/{x}/{y}.png"
            ],
            "tileSize": 256,
        }
    },
    "layers": [
        {
            "id": "imagery-tiles",
            "type": "raster",
            "source": "argenmap",
            "below": "waterway-label",
            "minzoom": 1,
            "maxzoom": 18,
        }
    ],
}

En el bloque anterior importamos todas las librerías que vamos a necesitar, importamos Point de shapely para poder usar las ubicaciones de las observaciones como puntos en un mapa, geopandas para manejar los datos de ubicación eficientemente y plotly para dibujar los mapas. Ahora obtenemos los datos:

In [4]:
if not ANIMALS_PATH.exists():
    import requests
    url = "https://archivos.linti.unlp.edu.ar/index.php/s/L2Gov0gmLDWX2rM/download"
    with open(ANIMALS_PATH, "wb") as f:
        f.write(requests.get(url).content)

animals_data = pd.read_csv(
    ANIMALS_PATH,
    encoding="utf-8",
    engine="c",
    usecols=COLS,
    dtype=TYPES,
)
gdf = gpd.read_file(AREAS_PATH)

En animals data tenemos las columnas de animales que nos interesan, la ubicación, los nombres, la imagen y su estado de la iucn, en caso de que no exista el archivo de animales (porque es muy grande para el git), se descarga de un repositorio, para cargar los datos de las áreas protegidas se usa geopandas que nos permitirá hacer operaciones como contains indicando si un punto se encuentra dentro de un área.

In [5]:
def is_within_area(row, gdf):
    return any(
        gdf.geometry.contains(Point(row["decimalLongitude"], row["decimalLatitude"]))
    )


def filter_data(animals_data, gdf):
    filtered_data = animals_data[animals_data.iucnRedListCategory.isin(["CR", "EN"])]
    filtered_data = filtered_data[
        filtered_data.apply(lambda row: is_within_area(row, gdf), axis=1)
    ]
    return filtered_data

Se filtran los datos para obtener los animales en peligro de extinción (animals_data.iucnRedListCategory.isin(["CR", "EN"])) y luego se usa la función is_within_area para filtrar las filas que tienen puntos de observaciones dentro de áreas protegidas.

In [6]:
def endangered_animals_map(filtered_data, gdf):
    fig = px.scatter_mapbox(
        lat=filtered_data["decimalLatitude"],
        lon=filtered_data["decimalLongitude"],
        hover_name=filtered_data["verbatimScientificName"],
        color_discrete_sequence=["red"],
    )
    fig.update_layout(
        mapbox=dict(
            style=ARGENMAP_STYLE, center=dict(lon=-59.08574, lat=-46.83348), zoom=5
        )
    )
    return fig

En este bloque se encuentra la función para generar el mapa de argentina con los datos, para esto se usa el scatter mapbox de plotly express, para esto se usa el estilo de argenmap definido más arriba y se colocan los puntos con los nombres científicos de los animales.

In [7]:
filtered_data = filter_data(animals_data, gdf)
endangered_animals_map(filtered_data, gdf).show()